In [ ]:
from scipy.spatial.distance import cdist
from scipy.stats import chi2
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
def calculate_histogram(image_path):
    """Calculează histograma culorilor normalizată (HSV) pentru o imagine."""
    try:
        # 1. Citeste imaginea
        image = cv2.imread(image_path)
        if image is None:
            print(f"Eroare: Nu s-a putut citi imaginea {image_path}. O sarim.")
            return None

        # 2. Convertește la spațiul de culoare HSV (mai bun pentru descrierea culorilor)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # 3. Calculează histograma (ex: H, S pe 8, 8 bins)
        # channels=[0, 1] pentru H și S; histSize=[8, 8] pentru 8 bin-uri pe fiecare
        # ranges=[0, 180, 0, 256] pentru H (0-180) și S (0-256)
        hist = cv2.calcHist([hsv], [0, 1], None, [8, 8], [0, 180, 0, 256])

        # 4. Normalizează histograma (pentru a fi insensibilă la mărimea imaginii)
        hist = cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX).flatten()
        return hist
    except Exception as e:
        print(f"A apărut o eroare la procesarea {image_path}: {e}")
        return None

In [ ]:
def find_similar_images(database_folder, query_image_path, x_top=5):
    """
    Găsește cele mai similare x imagini dintr-un folder la o imagine dată.

    :param database_folder: Calea către folderul cu imaginile bazei de date.
    :param query_image_path: Calea către imaginea de referință (de interes).
    :param x_top: Numărul de imagini similare de returnat.
    :return: O listă de tuple (cale_imagine, scor_similaritate).
    """
    
    # 1. Calculează histograma imaginii de interes (query)
    query_hist = calculate_histogram(query_image_path)
    if query_hist is None:
        print("\n🚫 Nu se poate continua, imaginea de interes nu a putut fi procesată.")
        return []

    # 2. Procesează imaginile din baza de date
    db_histograms = {}
    print(f"🔍 Procesez imaginile din directorul: {database_folder}...")

    # Extensii de imagine suportate
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    
    for filename in os.listdir(database_folder):
        # Ignoră imaginea de interes dacă se află în folderul bazei de date
        if filename == os.path.basename(query_image_path):
            continue
            
        if filename.lower().endswith(supported_extensions):
            path = os.path.join(database_folder, filename)
            hist = calculate_histogram(path)
            if hist is not None:
                db_histograms[path] = hist
    
    if not db_histograms:
        print("\n⚠️ Nu s-au găsit imagini valide în baza de date.")
        return []

    # 3. Compară histogramele
    results = {}
    
    # Metoda cv2.compareHist(hist1, hist2, method)
    # Metoda cv2.HISTCMP_CHISQR (Chi-Square) => Scorul mai MIC înseamnă similaritate MAI MARE.
    # Alternativ, se poate folosi corelația (cv2.HISTCMP_CORREL) unde scorul MAI MARE înseamnă similaritate MAI MARE.
    
    for path, hist in db_histograms.items():
        # Distanța Chi-Square
        distance = cv2.compareHist(query_hist, hist, cv2.HISTCMP_CHISQR)
        results[path] = distance

    # 4. Sortează rezultatele
    # Sortăm după valoarea distanței (scorul) în ordine ascendentă,
    # deoarece o distanță MAI MICĂ înseamnă o similaritate MAI MARE.
    sorted_results = sorted(results.items(), key=lambda item: item[1])

    # 5. Returnează primele X rezultate
    return sorted_results[:x_top]

In [ ]:
# --- Configurația Scriptului ---

# 🛑 ATENȚIE: Trebuie să adaptezi căile de mai jos la structura ta de fișiere!
# Exemplu de utilizare:
DATABASE_DIR = "test_custom"   # Ex: "./imagini_baza_de_date"
QUERY_IMAGE = "test_custom/face-3-center.jpg"    # Ex: "./imagini_query/caine_albastru.jpg"
TOP_X = 3

# Rulează funcția
similar_images = find_similar_images(DATABASE_DIR, QUERY_IMAGE, TOP_X)

## --- Afișarea Rezultatelor ---
print("\n" + "="*50)
print(f"✨ Cele mai similare {len(similar_images)} imagini cu '{os.path.basename(QUERY_IMAGE)}':")
print("="*50)

if similar_images:
    for i, (path, score) in enumerate(similar_images):
        # Scorul mai mic înseamnă similaritate mai mare (pentru Chi-Square)
        print(f"#{i+1}: {os.path.basename(path)}")
        print(f"  Scor similaritate (Chi-Square Dist.): {score:.4f}")
        # Afisare imagine (opțional)
        plt.figure(), plt.imshow(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB))
        plt.title(f"#{i+1}: {os.path.basename(path)}\nScor: {score:.4f}")
        plt.axis('off')
        plt.show()
        print("-" * 20)
else:
    print("Nicio imagine similară nu a fost găsită.")